In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))

In [2]:
import numpy as np
from tqdm.notebook import trange
from tqdm.notebook import tqdm

import model
import model_parameters as MP
from call_ledger import timer as T
#import dask.array as da

#np.get_printoptions()#["threshold"]
#np.set_printoptions(edgeitems=10, linewidth=150)

#np.random.seed(13)

## Mathematical Model

#### Sets
- C	Set of campaigns.
- U	Set of customers.
- H	Set of channels
- D	Set of planning days.
- I	Set of quota categories.
- P	Set of priority categories.


In [3]:
print(f"number of campaigns {MP.C}")
print(f"number of customers {MP.U}")
print(f"number of channels {MP.H}")
print(f"number of planning days {MP.D}")
print(f"number of quota categories {MP.I}")
print(f"number of priority categories {MP.P}")

number of campaigns 10
number of customers 1000
number of channels 3
number of planning days 7
number of quota categories 3
number of priority categories 10


#### Parameters

##### - eligibility
$$
e_{cu}\left\{\begin{array}\\
        1 & \mbox{if }  customer\ u\ is\ eligible\ for\ campaign\ c\\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [4]:
MP.e_cu_X

array([[[[1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        ...,

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        ...,

        [[1, 1, 1, ..., 1, 1, 1],


##### - quota categories
$$
q_{ic}\left\{\begin{array}\\
        1 & \mbox{if }  campaign\ c\ is\ a\ i^{th} type\ quota\ category\ campaign\ \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [5]:
MP.q_ic

array([[1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 1, 0, 0, 1, 1, 1],
       [0, 0, 1, 0, 1, 0, 0, 0, 1, 1]])

##### - priority categories
$$r_{cp}: Priority\ value\ of\ campaign\ c\ regarding\ priority\ type\ p\$$

In [6]:
MP.rp_c

array([44, 45, 89, 75, 17, 53, 53, 86, 53, 89])

##### - blokage
$$b: Communication\ limit\ per\ person\ for\ the\ whole\ period\$$

In [7]:
MP.b

7

##### - daily blokage
$$k: Communication\ limit\ per\ person\ at\ each\ day\$$

In [8]:
MP.k

3

##### - campaign blockage
$$l_c: Communication\ limit\ per\ person\ for\ campaign\ c\$$

In [9]:
MP.l_c

array([3, 2, 2, 2, 3, 3, 3, 2, 2, 4])

##### - quota limitations daily/weekly
$$
m_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\
$$
$$
n_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\ each\ day\
$$

In [10]:
(MP.m_i_X, MP.n_i)

(array([[5, 5, 5, ..., 5, 5, 5],
        [5, 5, 5, ..., 5, 5, 5],
        [5, 5, 5, ..., 5, 5, 5]]),
 array([3, 1, 2]))

#### - capacity for channel
$$
t_{h,d}: Capacity\ for\ channel\ h\ at\ day\ d.\
$$

In [11]:
MP.t_hd

array([[600., 700., 600., 700., 500., 600., 700.],
       [500., 500., 500., 600., 500., 600., 600.],
       [500., 700., 600., 600., 500., 500., 700.]])

# Model

#### Variables
$$
X_{cuhd}\left\{\begin{array}\\
        1 & \mbox{if } Campaign\ c\ will\ be\ sent\ to\ customer\ u\ through\ Channel\ h\ at\ Day\ d \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

## Maximize
$$\sum_{p \in P}\sum_{c \in C}\sum_{u \in U}\sum_{h \in H}\sum_{d \in D}\,X_{cuhd}\ r_{cp}$$

##### Binary variable (10)
$$
X_{cuhd} \in \{ 1,0 \},\hspace{35pt} \forall c \in C ,\forall u \in U,\forall d \in D, \forall h \in H
$$

In [12]:
X_cuhd = np.zeros((MP.C,MP.U,MP.H,MP.D), dtype='int')

## subject to

##### - eligibility (2)

$$
X_{cuhd}  \leq e_{cu},\hspace{35pt} \forall h \in H,\forall d \in D
$$

In [13]:
MP.X_eligibility

<function model_parameters.<lambda>(X)>

##### - use one channel (3)
$$
\sum_{h}X_{cuhd} \le 1,\hspace{35pt} \forall c \in C \, \forall u \in U,\forall d \in D
$$

In [14]:
MP.X_one_channel

<function model_parameters.<lambda>(X)>

##### - weekly communication limitation (4)
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in D} X_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$

In [15]:
MP.X_weekly_limitation

<function model_parameters.<lambda>(X)>

##### - daily communication limitation (5)
$$
\sum_{h \in H}\sum_{c \in C} X_{cuhd}\le k,\hspace{35pt} \forall u \in U, \forall d \in D
$$

In [16]:
MP.X_daily_limitation

<function model_parameters.<lambda>(X)>

##### - campaign communication limit(6)
$$
\sum_{d \in D}\sum_{h \in H} X_{cuhd}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$

In [17]:
MP.X_campaign_limitation

<function model_parameters.<lambda>(X)>

##### - weekly quota(7)
$$
\sum_{d \in D}\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$

In [18]:
MP.X_weekly_quota

<function model_parameters.<lambda>(X)>

##### - daily quota(8)
$$
\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le n_i,\hspace{35pt} \forall u \in U,\, \forall d \in D, \forall i \in I
$$

In [19]:
MP.X_daily_quota

<function model_parameters.<lambda>(X)>

##### Channel capacity (9)
$$
\sum_{c \in C}\sum_{u \in U}{X_{cuhd}}\le t_{hd},\hspace{35pt} \forall d \in D,\, \forall h \in H
$$

In [20]:
MP.X_channel_capacity

<function model_parameters.<lambda>(X)>

In [21]:
#The zero solution
X_cuhd = np.zeros((MP.C,MP.U,MP.H,MP.D), dtype='int')

## Model Definition

In [22]:
mdl = model.Model([
    model.Constraint('eligibility',MP.X_eligibility, ()),
    model.Constraint('channel_capacity',MP.X_channel_capacity, ()),
    model.Constraint('daily_limitation',MP.X_daily_limitation, ()),
    model.Constraint('weekly_limitation',MP.X_weekly_limitation, ()),
    model.Constraint('campaign_limitation',MP.X_campaign_limitation, ()),
    model.Constraint('daily_quota',MP.X_daily_quota, ()),
    model.Constraint('one_channel',MP.X_one_channel, ()),
    model.Constraint('weekly_quota',MP.X_weekly_quota, ())
], MP.objective_fn)

In [23]:
#c_i = 0
#u_i = 1
#h_i = 2
#d_i = 3

#mdl = model.Model([
#    model.Constraint('eligibility',MP.eligibility, (c_i, u_i, h_i, d_i,)),
#    model.Constraint('channel_capacity',MP.channel_capacity, (h_i, d_i,)),
#    model.Constraint('daily_limitation',MP.daily_limitation, (u_i, d_i,)),
#    model.Constraint('weekly_limitation',MP.weekly_limitation, (u_i,)),
#    model.Constraint('campaign_limitation',MP.campaign_limitation, (c_i, u_i,)),
#    model.Constraint('daily_quota',MP.daily_quota, (u_i, d_i,)),
#    model.Constraint('one_channel',MP.one_channel, (c_i, u_i, d_i,)),
#    model.Constraint('weekly_quota',MP.weekly_quota, (u_i,))
#], MP.objective_fn)

### Genetic Algorithm Meta Parameters 

In [24]:
import operator
import functools

INDIV_SIZE = (MP.C,MP.U,MP.H,MP.D)
INDIV_FLAT_SIZE = functools.reduce(operator.mul, INDIV_SIZE, 1)
POP_SIZE = 20#functools.reduce(operator.mul, INDIV_SIZE, 1)
GENERATION_SIZE = 100
CROSSOVER_PROB = 0.6
MUTATION_PROB = 1
MUTATION_RATE = 0.01
MUTATION_AMOUNT = int(MUTATION_RATE*INDIV_FLAT_SIZE)
NUMBER_OF_CROSSOVER_SECTION = 4

In [25]:
(POP_SIZE, INDIV_SIZE )

(20, (10, 1000, 3, 7))

### Initialize Population

In [26]:
def gen_greedy():
    for d in trange(MP.D, desc=f"Days Loop"):
        X_cuhd = np.zeros((MP.C,MP.U,MP.H,MP.D), dtype='int')
        for c in tqdm(np.argsort(-MP.rp_c), desc="Campaigns Loop"):
            for h in range(MP.H):#trange(H, desc=f"Channels Loop at Day-{d}, Campapaign-{c}"):
                for u in range(MP.U):#trange(U, desc=f"Users Loop On Campaign-{c}"):
                    X_cuhd[c,u,h,d]=1
                    if not mdl.execute(X_cuhd, (c, u, h, d)):
                        X_cuhd[c,u,h,d]=0
        yield X_cuhd

In [27]:
#X_X_cuhd=[z for z in gen_greedy()]

In [28]:
def fn_initialize_population(population_size, individual_size):
    #TODO take some kernel like matrix for randomization such as e_cu (the eligibility matrix)
    #np.stack([np.stack([MP.e_cu]*3, axis=2)]*7, axis=3)
    overall_size = tuple((population_size,)) + individual_size
    #return np.random.randint(2, size=overall_size)
    return np.zeros(overall_size)
#    return np.stack([X_X_cuhd[i%len(X_X_cuhd)] for i in range(population_size)])

### Fitness

In [29]:
import operator
import functools

def position_gene(j):
    j_d_m = j%MP.D
    j_d_r = j//MP.D
    j_h_m = j_d_r%MP.H
    j_h_r = j_d_r//MP.H
    j_u_m = j_h_r%MP.U
    j_u_r = j_h_r//MP.U
    j_c_m = j_u_r%MP.C
    j_c_r = j_u_r//MP.C
    return (j_c_m,j_u_m,j_h_m,j_d_m)

def fn_fitness_for_indiv(model, indiv):
    dims = indiv.shape
    rng = functools.reduce(operator.mul, dims, 1)
#    for j in range(rng):
#        if indiv[position_gene(j)]==1 and not model.execute(indiv, position_gene(j)):
#            return 0
    if not model.execute(indiv, ()):
        return 0
    return model.calc_value(indiv)

@T.timeit
def fn_fitness(model, population):
    return [(fn_fitness_for_indiv(model, indiv), indiv) for indiv in population]

### Selection functions

In [30]:
def __random_selection(size):
    return tuple(np.random.randint(low=0, high=POP_SIZE, size=size))

def __spin_wheel(fitnesses, sum_of_fitnesses):
    pin_point = np.random.randint(low=0, high=sum_of_fitnesses)
    fitness_index = 0
    for index, fitness in enumerate(fitnesses):
        pin_point = pin_point + fitness
        if pin_point > sum_of_fitnesses:
            return index
    return index

def roulettewheel_selection(population_with_fitness, size):
    fitnesses = np.array([f for (f,i) in population_with_fitness])
    sum_of_fitnesses = fitnesses.sum()
    if sum_of_fitnesses == 0:
        return __random_selection(size)
    else:
        return tuple([__spin_wheel(fitnesses, sum_of_fitnesses) for i in range(size)])

def __tournament_match(population_with_fitness):
    rivals = __random_selection(size=2)
    return rivals[np.argmax([population_with_fitness[i][0] for i in rivals])]
        
def tournament_selection(population_with_fitness, size):
    return tuple([__tournament_match(population_with_fitness) for i in range(size)])

@T.timeit
def select(selection_method, size, population_with_fitness, count):
#    print(f"selection with {selection_method.__name__}")
    for i in range(count):
        yield np.array([population_with_fitness[i][1] for i in selection_method(population_with_fitness, size)])

### Crossover

In [31]:
def __swap(sw):
    if sw ==0:
        return 1
    else:
        return 0

def __crossover(offspring, parent_index ,parents, crossover_sections):
    for s,e in crossover_sections:
        offspring[0].reshape(INDIV_FLAT_SIZE)[s:e] = parents[parent_index,__swap(1)].reshape(INDIV_FLAT_SIZE)[s:e]
        offspring[1].reshape(INDIV_FLAT_SIZE)[s:e] = parents[parent_index,__swap(0)].reshape(INDIV_FLAT_SIZE)[s:e]

@T.timeit
def crossover(parents):
    crossover_points = [p for p in np.sort(np.random.randint(low=1, high=INDIV_FLAT_SIZE, size =NUMBER_OF_CROSSOVER_SECTION))]
    crossover_sections = list(zip([0]+crossover_points,crossover_points+[INDIV_FLAT_SIZE]))
    offsprings = np.zeros(parents.shape, dtype='int')
    for index, offspring in enumerate(offsprings):
        __crossover(offspring, index ,parents, crossover_sections)
    return offsprings

### Mutation

In [32]:
def __mutate_allel(sw):
    indices_one = sw == 1
    indices_zero = sw == 0
    sw[indices_one] = 0 # replacing 1s with 0s
    sw[indices_zero] = 1 # replacing 0s with 1s
    return sw

@T.timeit
def mutation(generation):
    mutation_indicies = np.random.random(size=(POP_SIZE-2))<MUTATION_PROB
    mutants = generation[mutation_indicies]
    mutants = mutants.reshape((mutants.shape[0],INDIV_FLAT_SIZE))
    mutation_positions = np.random.randint(low=0, high=mutants.shape[1], 
                                           size=(mutants.shape[0], int(MUTATION_AMOUNT)))
    for mutant_index,mutation_position in enumerate(mutation_positions):
        mutants[mutant_index,mutation_position]=__mutate_allel(mutants[mutant_index,mutation_position])
    generation[mutation_indicies] = mutants.reshape(((mutants.shape[0],) + INDIV_SIZE))
    return generation

### A single Genetic Iteration

In [33]:
from multiprocessing.pool import ThreadPool
import time 
THREADS = 4
pool = ThreadPool(THREADS)

def genetic_iteration(population, selection_method, fitness_history):
    asyn_fitness_results=[pool.apply_async(fn_fitness, args=(mdl, popl)) 
                         for popl in np.split(population, THREADS)]
    fitness_results = sum([asyn_result.get() for asyn_result in asyn_fitness_results], [])
    population_with_fitness = sorted(fitness_results, key = lambda tup: tup[0], reverse=True)
    fitness_history.append(population_with_fitness[0][0])
    elit1 = population_with_fitness[0][1]
    elit2 = population_with_fitness[0][1]
    parents = np.stack([a for a in select(selection_method, 2, population_with_fitness, (POP_SIZE-2)//2)])
    next_generation = crossover(parents).reshape(((POP_SIZE-2,) + INDIV_SIZE))
    next_generation = mutation(next_generation)
    next_generation = np.append(next_generation, elit1)
    next_generation = np.append(next_generation, elit2)
    return next_generation.reshape(((POP_SIZE,) + INDIV_SIZE))

### Generation of generations

In [34]:
next_generation = fn_initialize_population(POP_SIZE, INDIV_SIZE)

In [35]:
type(next_generation)

numpy.ndarray

In [39]:
%%time

import operator
import functools
fitness_history = [0]
T.DURATION_STATS={}


INDIV_SIZE = (MP.C,MP.U,MP.H,MP.D)
INDIV_FLAT_SIZE = functools.reduce(operator.mul, INDIV_SIZE, 1)
POP_SIZE = 64#functools.reduce(operator.mul, INDIV_SIZE, 1)
GENERATION_SIZE = 20000
CROSSOVER_PROB = 0.6
MUTATION_PROB = 0.4
MUTATION_RATE = 0.50
NUMBER_OF_CROSSOVER_SECTION = 16
MUTATION_AMOUNT = int(MUTATION_RATE*INDIV_FLAT_SIZE)
MUTATION_AMOUNT = 3

#next_generation = fn_initialize_population(POP_SIZE, INDIV_SIZE)

for generation_index in trange(GENERATION_SIZE):
    next_generation = genetic_iteration(population=next_generation, 
                                        selection_method=tournament_selection, 
                                        #selection_method=roulettewheel_selection,
                                        fitness_history=fitness_history)
    if generation_index%100 == 0 and MUTATION_PROB > 0.05:
        MUTATION_PROB = MUTATION_PROB / 2
    if generation_index%100 == 0 and MUTATION_AMOUNT > 20:
        MUTATION_AMOUNT = MUTATION_AMOUNT - 10
    if generation_index%500 == 0:
        print((max(fitness_history), MUTATION_PROB, MUTATION_AMOUNT))
max(fitness_history)
#483312.0
#6779.0

  0%|          | 0/20000 [00:00<?, ?it/s]

(136973.0, 0.2, 3)
(150679.0, 0.05, 3)
(161645.0, 0.05, 3)
(171781.0, 0.05, 3)
(181038.0, 0.05, 3)
(190739.0, 0.05, 3)
(200861.0, 0.05, 3)
(208541.0, 0.05, 3)
(216457.0, 0.05, 3)
(223274.0, 0.05, 3)
(231937.0, 0.05, 3)
(238493.0, 0.05, 3)
(244218.0, 0.05, 3)
(250960.0, 0.05, 3)
(256423.0, 0.05, 3)
(261765.0, 0.05, 3)
(266669.0, 0.05, 3)
(270046.0, 0.05, 3)
(274161.0, 0.05, 3)
(278523.0, 0.05, 3)
(281117.0, 0.05, 3)
(284872.0, 0.05, 3)
(288545.0, 0.05, 3)
(291480.0, 0.05, 3)
(294579.0, 0.05, 3)
(297969.0, 0.05, 3)
(300343.0, 0.05, 3)
(302799.0, 0.05, 3)
(306081.0, 0.05, 3)
(308137.0, 0.05, 3)
(310420.0, 0.05, 3)
(313110.0, 0.05, 3)
(315584.0, 0.05, 3)
(317639.0, 0.05, 3)
(319334.0, 0.05, 3)
(320378.0, 0.05, 3)
(321992.0, 0.05, 3)
(324397.0, 0.05, 3)
(325706.0, 0.05, 3)
(326130.0, 0.05, 3)
CPU times: user 7h 7min 54s, sys: 1h 4min 58s, total: 8h 12min 52s
Wall time: 3h 46min 39s


326781.0

In [42]:
T.DURATION_STATS

{'fn_fitness.duration': 27498.602580009934,
 'fn_fitness.counter': 80000,
 'select.duration': 0.08868429192716576,
 'select.counter': 20000,
 'crossover.duration': 1260.4527139248512,
 'crossover.counter': 20000,
 'mutation.duration': 76.52411539115565,
 'mutation.counter': 20000}

In [41]:
fitness_history[4900:]

[230329.0,
 230329.0,
 230329.0,
 230329.0,
 230329.0,
 230471.0,
 230471.0,
 230471.0,
 230471.0,
 230471.0,
 230471.0,
 230471.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230521.0,
 230749.0,
 230749.0,
 230749.0,
 230749.0,
 230749.0,
 230749.0,
 230749.0,
 230749.0,
 230749.0,
 230749.0,
 230749.0,
 230932.0,
 230932.0,
 230932.0,
 230932.0,
 230932.0,
 230932.0,
 230932.0,
 230932.0,
 230932.0,
 230932.0,
 231118.0,
 231118.0,
 231118.0,
 231118.0,
 231118.0,
 231118.0,
 231118.0,
 231118.0,
 231310.0,
 231310.0,
 231310.0,
 231310.0,
 231310.0,
 231310.0,
 231310.0,
 231310.0,
 231310.0,
 231310.0,
 231310.0,
 231310.0,
 231310.0,
 231310.0,
 231310.0,
 231461.0,
 231461.0,
 231461.0,
 231461.0,
 231478.0,
 231478.0,
 231478.0,
 231478.0,
 231478.0,
 231478.0,
 231478.0,
 231659.0,